In [ ]:
import os
import pandas as pd
from pandas import json_normalize
from google.cloud import bigquery
from google.api_core.exceptions import GoogleAPIError
from tabulate import tabulate
from dotenv import load_dotenv

load_dotenv()
project_id = os.getenv('GOOGLE_PROJECT_ID')
key_path = os.getenv('GOOGLE_APPLICATION_CREDENTIALS')

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = key_path
client = bigquery.Client(project=project_id)
query = """
    SELECT *
    FROM `bigquery-public-data.ga4_obfuscated_sample_ecommerce.events_*`
    LIMIT 10
"""
query_job = client.query(query)
results = query_job.result()
rows = [dict(row) for row in results]
df = pd.DataFrame(rows)
if 'event_params' in df.columns:
    event_params = json_normalize(df['event_params'])
    df = df.drop('event_params', axis=1).join(event_params)
print(tabulate(df, headers='keys', tablefmt='psql'))